In [2]:
import pandas as pd
import numpy as np
import plotly.offline as pyo
import plotly.graph_objects as go
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State


## Basic

In [ ]:
app=dash.Dash()

colors={'background':'#111111', 'text':'#7FDBFF'}

app.layout=html.Div(children=[
    html.H1('Hello Dash!', style={'textAlign':'center',
                                 'color':colors['text']}),
    #html.Div('Dash: Web Dashboards!@!'),
    
    dcc.Graph(id='example', 
              figure={'data':[{'x':[1,2,3],
                               'y':[4,1,2],
                               'type':'bar', 
                               'name':'SF'},
                              {'x':[1,2,3],
                               'y':[2,4,5],
                               'type':'bar', 
                               'name':'ATL'}
                             ],
                      'layout': {
                          'plot_bgcolor':colors['background'],
                          'paper_bgcolor':colors['background'],
                          'font':{
                              'color':colors['text']
                          },
                          'title':'Bar Plots!'
                      }}
             )
], style={'backgroundColor':colors['background']})

if __name__=='__main__':
    app.run_server()
    
    


## Plotly 

In [ ]:
app=dash.Dash()
rx=np.random.randint(1,101,100)
ry=np.random.randint(1,101,100)
data=[go.Scatter(x=rx, y=ry, mode='markers', marker={'size':12,
                                                    'color':'rgb(51,205,153)',
                                                    'symbol':'pentagon',
                                                    'line': {'width':2}})]
layout=go.Layout(title='Hello Plot', xaxis={'title':'MY X'},yaxis={'title':'My Y'}, hovermode='closest')

fig=go.Figure(data=data, layout=layout)
pyo.plot(fig)


In [ ]:
app.layout=html.Div([dcc.Graph(id='my_scatter', 
                              figure={'data':data,
                                     'layout':layout}),
                   dcc.Graph(id='my_scatter2', 
                              figure={'data':data,
                                     'layout':layout})])

if __name__=='__main__':
    app.run_server()

## HTML Components

In [ ]:
app=dash.Dash()
app.layout=html.Div(['This is the otermost div!',
                     html.Div(['This is the inner div!'],
                              style={'color':'red',
                                     'border':'2px red solid'}),
                    html.Div(['This is the inner div!'],
                              style={'color':'blue',
                                     'border':'2px blue solid'})],
                    style={'color':'green',
                           'border':'2px green solid'},
                   )
if __name__=='__main__':
    app.run_server()

## Core Components

In [ ]:
app=dash.Dash()
app.layout=html.Div([
    html.Label('DropDown'),
    dcc.Dropdown(options=[{'label':'New Your City',
                          'value':'NYC'},
                         {'label':'Atlanta',
                          'value':'ATL'}],
                value='ATL'),
    html.Label('Slider'),
    dcc.Slider(min=-10,max=10, \
               step=0.5,value=0, marks={i: i for i in range(-10,10)}),
   
    html.P(html.Label('Some Radio Items')),
    dcc.RadioItems(options=[{'label':'New Your City',
                          'value':'NYC'},
                         {'label':'Atlanta',
                          'value':'ATL'}],
                value='ATL')
    
])
if __name__=='__main__':
    app.run_server()


### Callbacks

In [ ]:
app=dash.Dash()
app.layout=html.Div([
    dcc.Input(id='myid', value='Initial Text', type='text'),
    html.Div(id='mydiv',style={'color':'blue',
                                     'border':'2px blue solid'})
])
@app.callback(Output(component_id='mydiv', component_property='children'),
             [Input(component_id='myid', component_property='value')])
def update_ouput_div(input_value):
    return "You entered {}".format(input_value)

if __name__=='__main__':
    app.run_server()

In [ ]:
df=pd.read_csv('https://raw.githubusercontent.com/Pierian-Data/Plotly-Dashboards-with-Dash/master/Data/gapminderDataFiveYear.csv')

In [ ]:
app=dash.Dash()
year_options=[]
for year in df['year'].unique():
    year_options.append({'label':str(year),'value':year})
                         
                         
app.layout=html.Div([
    dcc.Graph(id='graph'),
    dcc.Dropdown(id='year-picker',
                options=year_options)
])

####----------------------------

@app.callback(Output('graph','figure'), 
             [Input('year-picker', 'value')])
def update_figure(selected_year):
    filtered_df=df[df['year']==selected_year]
    traces=[]
    for continent in filtered_df['continent'].unique():
        df_by_continent=filtered_df[filtered_df['continent']==continent]
        traces.append(go.Scatter(
        x=df_by_continent['gdpPercap'],
        y=df_by_continent['lifeExp'],
        mode='markers',
        opacity=.7,
        marker={'size':15},
        name=continent))
        
    return {'data':traces,
           'layout':go.Layout(title='Hello Plot', xaxis={'title':'MY X'},yaxis={'title':'My Y'}, hovermode='closest')
}


####----------------------------
if __name__=='__main__':
    app.run_server()

In [ ]:
df=pd.read_csv('https://raw.githubusercontent.com/Pierian-Data/Plotly-Dashboards-with-Dash/master/Data/mpg.csv')

app=dash.Dash()
features=df.columns.to_list()
print(features)

app.layout=html.Div([
    
    html.Div([
        dcc.Dropdown(id='xaxis',
                    options=[{'label':i,
                              'value':i} for i in features],
                     value='displacement')
    ], style={'width':'48%', 'display':'inline-block'}),
    
    html.Div([
        dcc.Dropdown(id='yaxis',
                    options=[{'label':i,
                              'value':i} for i in features],
                    value='mpg')
    ], style={'width':'48%', 'display':'inline-block'}), 
    dcc.Graph(id='feature-graphic')
], style={'padding':10})

@app.callback(Output('feature-graphic','figure'),
             [Input('xaxis','value'),
              Input('yaxis','value')])
def update_graph(xaxis_name,yaxis_name):
    return{'data':[go.Scatter(x=df[xaxis_name],
                              y=df[yaxis_name], 
                              text=df['name'],
                              mode='markers',
                              marker={'size':15,
                                     'opacity':.5,
                                     'line':{'width':.5,
                                            'color':'white'}})],'layout':go.Layout(title='My MPG DASH',
                                                                                   xaxis={'title':xaxis_name},
                                                                                   yaxis={'title':yaxis_name},
                                                                                   hovermode='closest')}





####----------------------------
if __name__=='__main__':
    app.run_server()



In [ ]:
###NOT FULLY WORKING###

df=pd.read_csv('https://raw.githubusercontent.com/Pierian-Data/Plotly-Dashboards-with-Dash/master/Data/wheels.csv')

app=dash.Dash()

app.layout=html.Div([
    dcc.RadioItems(id='wheels',
                  options=[{'label':i, 'value':i} for i in df['wheels'].unique()],
                  value=1),
    html.Div(id='wheels-output'),
    html.Hr(),
    dcc.RadioItems(id='colors',
                   options=[{'label':i, 'value':i} for i in df['color'].unique()],
                   value='blue'),
    html.Div(id='colors-output')
], style={'fontFamily':'helvetica', 'fontSize':18})


@app.callback(Output('wheels-output','children'),
                    [Input('wheels', 'value')])
def callback_a(wheels_value):
    return "You chose {}".format(wheels_values)

@app.callback(Output('colors-output','children'),
                    [Input('colors', 'value')])
def callback_b(colors_value):
    return "You chose {}".format(color_values)



####----------------------------
if __name__=='__main__':
    app.run_server()

# State

In [ ]:
app=dash.Dash()
app.layout=html.Div([
    dcc.Input(id='number-in', value=1, style={'fontSize':24}),
    html.Button(id='submit-button', n_clicks=0, 
                children='Submit here',
                style={'fontSize':24}
               ),
                
               
    html.H1(id='number-out')
])




@app.callback(Output('number-out', 'children'),
             [Input('submit-button', 'n_clicks')],
             [State('number-in', 'value')])

def output(n_clicks, number):
    return "{} was typed and button was clicked {} times".format(number,n_clicks)

if __name__=='__main__':
    app.run_server()

# Select

In [ ]:
#######
# This makes a 3x3 scatterplot of wheels.csv, and sends
# the results of a selection to the screen as a JSON object.
######
# import dash
# import dash_core_components as dcc
# import dash_html_components as html
# from dash.dependencies import Input, Output
# import plotly.graph_objs as go
# import pandas as pd
# import json

app = dash.Dash()

df=pd.read_csv('https://raw.githubusercontent.com/Pierian-Data/Plotly-Dashboards-with-Dash/master/Data/wheels.csv')


app.layout = html.Div([
    html.Div([
    dcc.Graph(
        id='wheels-plot',
        figure={
            'data': [
                go.Scatter(
                    x = df['color'],
                    y = df['wheels'],
                    dy = 1,
                    mode = 'markers',
                    marker = {
                        'size': 12,
                        'color': 'rgb(51,204,153)',
                        'line': {'width': 2}
                        }
                )
            ],
            'layout': go.Layout(
                title = 'Wheels & Colors Scatterplot',
                xaxis = {'title': 'Color'},
                yaxis = {'title': '# of Wheels','nticks':3},
                hovermode='closest'
            )
        }
    )], style={'width':'30%', 'display':'inline-block'}),

    html.Div([
    html.Pre(id='selection', style={'paddingTop':25})
    ], style={'width':'30%', 'display':'inline-block', 'verticalAlign':'top'})
])

@app.callback(
    Output('selection', 'children'),
    [Input('wheels-plot', 'selsectedData')])
def callback_image(selectedData):
    return json.dumps(selectedData, indent=2)

if __name__ == '__main__':
    app.run_server()

In [ ]:
app = dash.Dash()

# create x and y arrays
np.random.seed(10)          # for reproducible results
x1 = np.linspace(0.1,5,50)  # left half
x2 = np.linspace(5.1,10,50) # right half
y = np.random.randint(0,50,50) # 50 random points

# create three "half DataFrames"
df1 = pd.DataFrame({'x': x1, 'y': y})
df2 = pd.DataFrame({'x': x1, 'y': y})
df3 = pd.DataFrame({'x': x2, 'y': y})

# combine them into one DataFrame (df1 and df2 points overlap!)
df = pd.concat([df1,df2,df3])

app.layout = html.Div([
    html.Div([
    dcc.Graph(
        id='plot',
        figure={
            'data': [
                go.Scatter(
                    x = df['x'],
                    y = df['y'],
                    mode = 'markers'
                )
            ],
            'layout': go.Layout(
                title = 'Random Scatterplot',
                hovermode='closest'
            )
        }
    )], style={'width':'30%', 'display':'inline-block'}),

    html.Div([
    html.H1(id='density', style={'paddingTop':25})
    ], style={'width':'30%', 'display':'inline-block', 'verticalAlign':'top'})
])

@app.callback(
    Output('density', 'children'),
    [Input('plot', 'selectedData')])
def find_density(selectedData):
    pts = len(selectedData['points'])
    rng_or_lp = list(selectedData.keys())
    rng_or_lp.remove('points')
    max_x = max(selectedData[rng_or_lp[0]]['x'])
    min_x = min(selectedData[rng_or_lp[0]]['x'])
    max_y = max(selectedData[rng_or_lp[0]]['y'])
    min_y = min(selectedData[rng_or_lp[0]]['y'])
    area = (max_x-min_x)*(max_y-min_y)
    d = pts/area
    return 'Density = {:.2f}'.format(d)

if __name__ == '__main__':
    app.run_server()


# Update Graph Interacation

In [3]:
df=pd.read_csv('https://raw.githubusercontent.com/Pierian-Data/Plotly-Dashboards-with-Dash/master/Data/mpg.csv')
#add jitter
df['year']=np.random.randint(-4,5,len(df))*.1 +df['model_year']

app=dash.Dash()

app.layout=html.Div([
    html.Div([ dcc.Graph(id='mpg_scatter', figure={
        'data':[go.Scatter(x=df['year']+1900,
                          y=df['mpg'],
                          text=df['name'],
                          hoverinfo='text',
                          mode='markers')], 
        'layout':go.Layout(title='MPG Data',
                          xaxis={'title':'Modle Year'},
                          yaxis={'title':'MPG'},
                          hovermode='closest')
    })
             ], style={'width':'50%', 'display':'inline-block'}),
    
    html.Div([dcc.Graph(id='mpg-line',
                       figure={'data':[go.Scatter(x=[0,1],
                                                  y=[0,1],
                                                 mode='lines')],
                              'layout':go.Layout(title='Acceleration',
                                                margin={'l':0})})
        
    ],style={'width':'40%', 'display':'inline-block'}),
    
    html.Div([dcc.Markdown(id='mpg_stats')
        
    ],style={'width':'10%','height':'50%', 'display':'inline-block'})

    
])



#Output1 Graph
@app.callback(Output('mpg-line', 'figure'),
             [Input('mpg_scatter', 'hoverData')])

def callback_graph(hoverData):
    print('-------------------------------------------')
    print('-------------------------------------------')
    print('-------------------------------------------')
    
    print(hoverData)
    print(hoverData['points'])
    print(hoverData['points'][0])
    print(hoverData['points'][0]['pointIndex'])
    print('-------------------------------------------')
    print('-------------------------------------------')
    print('-------------------------------------------')
    v_index=hoverData['points'][0]['pointIndex']
    figure={'data':[go.Scatter(x=[0,1],
                              y=[0,60/df.iloc[v_index]['acceleration']],
                              mode='lines',
                              line={'width':2*df.iloc[v_index]['cylinders']})],
            'layout': go.Layout(title=df.iloc[v_index]['name'],
                                xaxis={'visible':False},
                                yaxis={'visible':False, 'range':[0,60/df['acceleration'].min()]},
                                margin={'l':0},
                                height=300)
           }
    return figure

#Output 2 Markdown
@app.callback(Output('mpg_stats', 'children'),
             [Input('mpg_scatter', 'hoverData')])

def callback_stats(hoverData):
    v_index=hoverData['points'][0]['pointIndex']
    stats="""
    {} cylinders
    {} cc Displacement
    0 to 60mph in {} seconds
    """.format(df.iloc[v_index]['cylinders'],
               df.iloc[v_index]['displacement'],
               df.iloc[v_index]['acceleration']
              )
    print(stats)
    return stats
    
    






if __name__ == '__main__':
    app.run_server()

    
    
    
    
    


Dash is running on http://127.0.0.1:8050/

 in production, use a production WSGI server like gunicorn instead.

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [15/Jul/2020 16:11:55] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Jul/2020 16:11:55] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [15/Jul/2020 16:11:55] "GET /_dash-layout HTTP/1.1" 200 -


-------------------------------------------
-------------------------------------------
-------------------------------------------
None
Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/wp_alex/opt/anaconda3/envs/myenv2/lib/python3.6/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/wp_alex/opt/anaconda3/envs/myenv2/lib/python3.6/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/wp_alex/opt/anaconda3/envs/myenv2/lib/python3.6/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/wp_alex/opt/anaconda3/envs/myenv2/lib/python3.6/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/Users/wp_alex/opt/anaconda3/envs/myenv2/lib/python3.6/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()


127.0.0.1 - - [15/Jul/2020 16:11:55] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/wp_alex/opt/anaconda3/envs/myenv2/lib/python3.6/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/wp_alex/opt/anaconda3/envs/myenv2/lib/python3.6/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/wp_alex/opt/anaconda3/envs/myenv2/lib/python3.6/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/wp_alex/opt/anaconda3/envs/myenv2/lib/python3.6/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/Users/wp_alex/opt/anaconda3/envs/myenv2/lib/python3.6/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/wp_alex/opt/anaconda3/envs/myenv2/lib/python3.6/site-packages/flask/app.py", line 1936, in dispatch_request
    return sel

...

-------------------------------------------
-------------------------------------------
-------------------------------------------
    6 cylinders
    225.0 cc Displacement
    0 to 60mph in 17.7 seconds
    

{'points': [{'curveNumber': 0, 'pointNumber': 227, 'pointIndex': 227, 'x': 1977.1, 'y': 19, 'text': 'plymouth volare custom'}]}
[{'curveNumber': 0, 'pointNumber': 227, 'pointIndex': 227, 'x': 1977.1, 'y': 19, 'text': 'plymouth volare custom'}]
{'curveNumber': 0, 'pointNumber': 227, 'pointIndex': 227, 'x': 1977.1, 'y': 19, 'text': 'plymouth volare custom'}
227
-------------------------------------------
-------------------------------------------
-------------------------------------------
-------------------------------------------
-------------------------------------------
-------------------------------------------
{'points': [{'curveNumber': 0, 'pointNumber': 202, 'pointIndex': 202, 'x': 1976, 'y': 17.5, 'text': 'amc pacer d/l'}]}
[{'curveNumber': 0, 'pointNumber': 202, 'poi

127.0.0.1 - - [15/Jul/2020 16:11:56] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Jul/2020 16:11:56] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Jul/2020 16:11:56] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Jul/2020 16:11:56] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Jul/2020 16:11:56] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Jul/2020 16:11:56] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Jul/2020 16:11:56] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Jul/2020 16:11:56] "POST /_dash-update-component HTTP/1.1" 200 -



    6 cylinders
    258.0 cc Displacement
    0 to 60mph in 17.8 seconds
    
-------------------------------------------
-------------------------------------------
-------------------------------------------
{'points': [{'curveNumber': 0, 'pointNumber': 160, 'pointIndex': 160, 'x': 1975.2, 'y': 17, 'text': 'buick century'}]}
[{'curveNumber': 0, 'pointNumber': 160, 'pointIndex': 160, 'x': 1975.2, 'y': 17, 'text': 'buick century'}]
{'curveNumber': 0, 'pointNumber': 160, 'pointIndex': 160, 'x': 1975.2, 'y': 17, 'text': 'buick century'}
160
-------------------------------------------
-------------------------------------------
-------------------------------------------

    6 cylinders
    231.0 cc Displacement
    0 to 60mph in 21.0 seconds
    
-------------------------------------------
-------------------------------------------
-------------------------------------------
{'points': [{'curveNumber': 0, 'pointNumber': 135, 'pointIndex': 135, 'x': 1974, 'y': 18, 'text': 'plymouth sat

127.0.0.1 - - [15/Jul/2020 16:11:56] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Jul/2020 16:11:56] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Jul/2020 16:11:56] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Jul/2020 16:11:56] "POST /_dash-update-component HTTP/1.1" 200 -


-------------------------------------------
    4 cylinders
    98.0 cc Displacement
    0 to 60mph in 15.5 seconds
    

-------------------------------------------
-------------------------------------------
{'points': [{'curveNumber': 0, 'pointNumber': 114, 'pointIndex': 114, 'x': 1973.1, 'y': 26, 'text': 'fiat 124 sport coupe'}]}
[{'curveNumber': 0, 'pointNumber': 114, 'pointIndex': 114, 'x': 1973.1, 'y': 26, 'text': 'fiat 124 sport coupe'}]
{'curveNumber': 0, 'pointNumber': 114, 'pointIndex': 114, 'x': 1973.1, 'y': 26, 'text': 'fiat 124 sport coupe'}
114
-------------------------------------------
-------------------------------------------
-------------------------------------------
-------------------------------------------
-------------------------------------------
-------------------------------------------
{'points': [{'curveNumber': 0, 'pointNumber': 131, 'pointIndex': 131, 'x': 1973.9, 'y': 32, 'text': 'toyota corolla 1200'}]}
[{'curveNumber': 0, 'pointNumber': 131, 'poin

In [ ]:
df['year'] = df['model_year'] + np.random.randint(-4,5,len(df))*0.10

app.layout = html.Div([
    html.Div([   # this Div contains our scatter plot
    dcc.Graph(
        id='mpg_scatter',
        figure={
            'data': [go.Scatter(
                x = df['year']+1900,  # our "jittered" data
                y = df['mpg'],
                text = df['name'],
                hoverinfo = 'text',
                mode = 'markers'
            )],
            'layout': go.Layout(
                title = 'mpg.csv dataset',
                xaxis = {'title': 'model year'},
                yaxis = {'title': 'miles per gallon'},
                hovermode='closest'
            )
        }
    )], style={'width':'50%','display':'inline-block'}),
    html.Div([  # this Div contains our output graph and vehicle stats
    dcc.Graph(
        id='mpg_line',
        figure={
            'data': [go.Scatter(
                x = [0,1],
                y = [0,1],
                mode = 'lines'
            )],
            'layout': go.Layout(
                title = 'acceleration',
                margin = {'l':0}
            )
        }
    ),
    dcc.Markdown(
        id='mpg_stats'
    )
    ], style={'width':'20%', 'height':'50%','display':'inline-block'})
])

@app.callback(
    Output('mpg_line', 'figure'),
    [Input('mpg_scatter', 'hoverData')])
def callback_graph(hoverData):
    v_index = hoverData['points'][0]['pointIndex']
    fig = {
        'data': [go.Scatter(
            x = [0,1],
            y = [0,60/df.iloc[v_index]['acceleration']],
            mode='lines',
            line={'width':2*df.iloc[v_index]['cylinders']}
        )],
        'layout': go.Layout(
            title = df.iloc[v_index]['name'],
            xaxis = {'visible':False},
            yaxis = {'visible':False, 'range':[0,60/df['acceleration'].min()]},
            margin = {'l':0},
            height = 300
        )
    }
    return fig

@app.callback(
    Output('mpg_stats', 'children'),
    [Input('mpg_scatter', 'hoverData')])
def callback_stats(hoverData):
    v_index = hoverData['points'][0]['pointIndex']
    stats = """
        {} cylinders
        {}cc displacement
        0 to 60mph in {} seconds
        """.format(df.iloc[v_index]['cylinders'],
            df.iloc[v_index]['displacement'],
            df.iloc[v_index]['acceleration'])
    return stats

if __name__ == '__main__':
    app.run_server()